In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
!pip install pandas numpy matplotlib plotly scikit-learn joblib sqlalchemy
import pandas as pd, numpy as np
from pathlib import Path
DATA_DIR = Path('/content/data')
DATA_DIR.mkdir(parents=True, exist_ok=True)


In [16]:
df = pd.read_csv('/content/drive/MyDrive/Customer_Subscription_And_Transaction_Details.csv', dtype=str)
df.shape
df.head()
df.info()
df.nunique()
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18106 entries, 0 to 18105
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   cust_id             18106 non-null  object
 1   transaction_type    18106 non-null  object
 2   transaction_date    18106 non-null  object
 3   subscription_type   18106 non-null  object
 4   subscription_price  18106 non-null  object
 5   customer_gender     18106 non-null  object
 6   age_group           18106 non-null  object
 7   customer_country    18106 non-null  object
 8   referral_type       18106 non-null  object
dtypes: object(9)
memory usage: 1.2+ MB


,0
cust_id,0
transaction_type,0
transaction_date,0
subscription_type,0
subscription_price,0
customer_gender,0
age_group,0
customer_country,0
referral_type,0


In [17]:
df['transaction_date'] = pd.to_datetime(df['transaction_date'], format='%d-%m-%Y', errors='coerce')
df['year'] = df['transaction_date'].dt.year
df['month'] = df['transaction_date'].dt.to_period('M').astype(str)


In [18]:
df['transaction_type'] = df['transaction_type'].str.lower().str.strip()
df['subscription_type'] = df['subscription_type'].str.upper().str.strip()
df['referral_type'] = df['referral_type'].str.lower().str.strip()


In [19]:
df[df.duplicated(subset=['cust_id','transaction_date','transaction_type','subscription_type'])]
df[df['subscription_price'].astype(float) <= 0]


,cust_id,transaction_type,transaction_date,subscription_type,subscription_price,customer_gender,age_group,customer_country,referral_type,year,month


In [20]:
df['cust_id'] = df['cust_id'].astype(int)

first_tx = df.groupby('cust_id')['transaction_date'].min().reset_index().rename(columns={'transaction_date':'first_tx'})
df = df.merge(first_tx, on='cust_id', how='left')
df['cohort_month'] = df['first_tx'].dt.to_period('M').astype(str)

plan_map = {p:i+1 for i,p in enumerate(df['subscription_type'].unique())}
df['plan_id'] = df['subscription_type'].map(plan_map)


In [21]:

df['billing_month'] = df['transaction_date'].dt.to_period('M').dt.to_timestamp()
monthly_rev = df[df['transaction_type'].isin(['initial','renewal'])].groupby('billing_month')['subscription_price'].sum().reset_index()
monthly_rev.to_csv('monthly_revenue.csv', index=False)


In [22]:

df['is_churn'] = df['transaction_type']=='cancellation'
churn_by_user = df.groupby('cust_id')['is_churn'].any().reset_index()
churn_by_user.to_csv('churn_by_user.csv', index=False)


In [23]:
tx_agg = df.groupby('cust_id').agg(
    renewal_count = ('transaction_type', lambda s: (s=='renewal').sum()),
    first_tx = ('transaction_date','min'),
    last_tx = ('transaction_date','max'),
    total_revenue = ('subscription_price','sum')
).reset_index()
tx_agg['tenure_days'] = (tx_agg['last_tx'] - tx_agg['first_tx']).dt.days
tx_agg.to_csv('customer_features.csv', index=False)


In [24]:

df['subscription_price'] = df['subscription_price'].astype(float)


df['billing_month'] = df['transaction_date'].dt.to_period('M').dt.to_timestamp()
monthly_rev = df[df['transaction_type'].isin(['initial','renewal'])].groupby('billing_month')['subscription_price'].sum().reset_index()


ltv = df.groupby('cust_id')['subscription_price'].sum().reset_index().rename(columns={'subscription_price':'ltv'})
active_users_month = df.groupby('month')['cust_id'].nunique()
arpu_by_month = monthly_rev.assign(arpu = monthly_rev['subscription_price'] / active_users_month.loc[monthly_rev['billing_month'].dt.to_period('M').astype(str)].values)

ltv.to_csv('user_ltv.csv', index=False)
arpu_by_month.to_csv('arpu_by_month.csv', index=False)

In [25]:

df['cohort_month'] = df.groupby('cust_id')['transaction_date'].transform('min').dt.to_period('M')
df['active_month'] = df['transaction_date'].dt.to_period('M')
cohort = df.groupby(['cohort_month','active_month'])['cust_id'].nunique().reset_index()

cohort['months_since'] = (cohort['active_month'].dt.year - cohort['cohort_month'].dt.year) * 12 + \
                         (cohort['active_month'].dt.month - cohort['cohort_month'].dt.month)
cohort.to_csv('cohort_long.csv', index=False)